<a href="https://colab.research.google.com/github/m-allahkhani/flower-clustering-project/blob/main/project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

maryamm allahkhani
shaghayegh shafiee

In [1]:
import pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from scipy.spatial import ConvexHull
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import random
import cv2
from statistics import mode
import math
# from google.colab import drive
import pandas as pd
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, accuracy_score



In [ ]:
# import tensorflow as tf
# print("GPU Available ",tf.test.gpu_device_name())

GPU Available  /device:GPU:0


In [2]:
def load_dataset():
    image_pickle_file_path = 'images.pkl'
    label_pickle_file_path = 'label.pkl'
    # drive.mount('/content/drive')
    with open(image_pickle_file_path, 'rb') as file:
        images = pickle.load(file)

    with open(label_pickle_file_path, 'rb') as file:
        labels = pickle.load(file)
    # images = np.load('drive/MyDrive/Colab Notebooks/images.pkl', allow_pickle = True)
    # labels = np.load('drive/MyDrive/Colab Notebooks/label.pkl', allow_pickle = True)
    features = []
    # pixel_locations = []
    # hsv_values = []
    for image in images:
        n, m, _ = image.shape
        indices = np.indices((n, m))

        pixel_img = np.column_stack((indices[0].ravel(), indices[1].ravel()))
        # pixel_locations.append(pixel_img)

        hsv_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)

        hsv = hsv_image.reshape(n*m, -1)
        # hsv_values.append(hsv)
        feature_image = np.column_stack((hsv,pixel_img))
        features.append(feature_image)
    # return images, labels, pixel_locations, hsv_values , features
    return images ,labels, features

In [3]:
# Classify the datapoints with the Random Forest Classifier
def classify(feature_vectors, labels):


    feature_vectors = np.array(feature_vectors)
    scaler = StandardScaler()
    normalized_features = scaler.fit_transform(feature_vectors[:, :])

    weighted_features = np.empty([len(normalized_features), 8])
    weighted_features[:, 0] = normalized_features[:, 0] * 3
    weighted_features[:, 1] = normalized_features[:, 1] * 2
    weighted_features[:, 2] = normalized_features[:, 2] * 2
    weighted_features[:, 3] = normalized_features[:, 3] * 2
    weighted_features[:, 4] = normalized_features[:, 4] * 2
    weighted_features[:, 5] = normalized_features[:, 5] * 1
    weighted_features[:, 6] = normalized_features[:, 6] * 1
    weighted_features[:, 7] = normalized_features[:, 7] * 1

    test_size = 0.2
    X_train, X_test, y_train, y_test = train_test_split(weighted_features, labels, test_size=test_size, random_state=42)

    clf = RandomForestClassifier(n_estimators=100, random_state=42)

    clf.fit(X_train, y_train)

    y_pred = clf.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy: {accuracy * 100:.2f}%")
    return y_test,y_pred,accuracy

In [4]:

def extract_flower_parts(features):
    numberCluster = 3
    scaler = StandardScaler()
    width = np.max(features[:, 3])#center of the image
    height = np.max(features[:, 4])
    center_x =  width / 2
    center_y = height / 2

    #  horizontal and vertical space each pixel and the center
    horizontal_space = np.abs(features[:, 3] - center_x)
    vertical_space = np.abs(features[:, 4] - center_y)
    horizontal_space_normalized = scaler.fit_transform(horizontal_space.reshape(-1, 1))
    vertical_space_normalized = scaler.fit_transform(vertical_space.reshape(-1, 1))


    num_components = 2    # number of components for GMM
    original_hue_values = features[:, 0].copy()
    gmm = GaussianMixture(n_components=num_components)
    gmm.fit(original_hue_values.reshape(-1, 1))

    # Assign each pixel to the most likely cluster based on its hue value
    cluster_assignments = gmm.predict(original_hue_values.reshape(-1, 1))
    arranged_hue_values = gmm.means_[cluster_assignments]
    normalized_features = scaler.fit_transform(features[:, 1:5])
    normalized_hue_values =scaler.fit_transform(arranged_hue_values.reshape(-1, 1))
    weighted_features = np.empty([len(normalized_features), 7])
    weighted_features[:, 0] = normalized_hue_values.flatten() * 12
    weighted_features[:, 1] = normalized_features[:, 0] * 5
    weighted_features[:, 2] = normalized_features[:, 1] * 8
    weighted_features[:, 3] = normalized_features[:, 2] * 6
    weighted_features[:, 4] = normalized_features[:, 3] * 6
    weighted_features[:, 5] = horizontal_space_normalized.flatten() * 3  #  horizontal space
    weighted_features[:, 6] = vertical_space_normalized.flatten() * 3  #  vertical space

    kmeans = KMeans(n_clusters=numberCluster)
    kmeans.fit(weighted_features)
    cluster_labels = kmeans.predict(weighted_features)
    # cluster_centers = kmeans.cluster_centers_


    labeled_pixels = []
    pixels = []
    for i in range(len(features)):
        pixel = [int(features[i][3]), int(features[i][4]), cluster_labels[i]]
        labeled_pixels.append(pixel)
        pixel2 = [int(features[i][0]), int(features[i][1]), int(features[i][2]), int(features[i][3]),int(features[i][4]), cluster_labels[i]]
        pixels.append(pixel2)

    clusters = [[] for _ in range(numberCluster)]
    for i, pixel in enumerate(pixels):
        cluster_label = pixel[5]
        clusters[cluster_label].append([features[i][0], features[i][1], features[i][2], features[i][3],features[i][4], cluster_label])

    return labeled_pixels,clusters,width,height,len(features)

In [5]:
def show_cluster_image2(pixels, target_label,max_row,max_col): #image_width and height


    # Initialize an empty image with dimensions based on the maximum row and column values
    image = np.zeros((max_row + 1, max_col + 1, 3), dtype=np.uint8)

    # Assign unique colors to each cluster label excluding target label
    label_colors = {}
    for pixel in pixels:
        _, _, label = pixel
        if label not in label_colors:
            if label == target_label:
                label_colors[label] = np.array([255, 0, 0], dtype=np.uint8)
            else:
                while True:
                    color = np.random.randint(0, 256, size=3, dtype=np.uint8)
                    if not np.array_equal(color, [255, 0, 0]): # Avoid red
                        label_colors[label] = color
                        break

    # Fill the image pixels with the corresponding cluster label color
    for pixel in pixels:
        row, col, label = pixel
        color = label_colors[label]
        image[row, col] = color
    # Display the image
    plt.imshow(image)
    plt.axis('off')
    plt.show()


In [6]:
def show_image(image):
    plt.imshow(image)
    plt.axis('off')
    plt.show()

In [7]:
def show_cluster_image(pixels,max_row,max_col): #image_width and height


    # Initialize an empty image with dimensions based on the maximum row and column values
    image = np.zeros((max_row + 1, max_col + 1, 3), dtype=np.uint8)

    # Assign unique colors to each cluster label
    label_colors = {}
    for pixel in pixels:
        _, _, label = pixel
        if label not in label_colors:
            label_colors[label] = np.random.randint(0, 256, size=3, dtype=np.uint8)

    # Fill the image pixels with the corresponding cluster label color
    for pixel in pixels:
        row, col, label = pixel
        color = label_colors[label]
        image[row, col] = color

    # Display the image
    plt.imshow(image)
    plt.axis('off')
    plt.show()

In [11]:
def find_label(num):
    images2,labels,features = load_dataset()
    for i in range(0,len(images2)):
        if(labels[i]==num):
            show_image(images2[i])
            pixel_labels2,clusters,width,height,points_num= extract_flower_parts(features[i])
            flower_index = find_flower_cluster(clusters,width,height,points_num)
            show_cluster_image2(pixel_labels2,clusters[flower_index][0][5],width,height)


In [8]:

def calculate_hue_change_coefficients(pixels, image_width, image_height):
    x = np.array([pixel[4] for pixel in pixels])  # Column indices
    y = np.array([pixel[3] for pixel in pixels])  # Row indices
    hue = np.array([pixel[0] for pixel in pixels])  # Hue values

    # distance to center
    distance_to_center = np.sqrt((y - image_height/2)**2 + (x - image_width/2)**2)
    max_distance = np.sqrt((image_height/2)**2 + (image_width/2)**2)
    normalized_distance = distance_to_center / max_distance

    # Left side
    left_indices = x < image_width/2
    a1 = np.sum((hue[left_indices] - np.mean(hue)) * normalized_distance[left_indices]) / np.sum(normalized_distance[left_indices]**2)

    # Right side
    right_indices = x >= image_width/2
    a2 = np.sum((hue[right_indices] - np.mean(hue)) * normalized_distance[right_indices]) / np.sum(normalized_distance[right_indices]**2)

    # Top side
    top_indices = y < image_height/2
    a3 = np.sum((hue[top_indices] - np.mean(hue)) * normalized_distance[top_indices]) / np.sum(normalized_distance[top_indices]**2)

    # Bottom side
    bottom_indices = y >= image_height/2
    a4 = np.sum((hue[bottom_indices] - np.mean(hue)) * normalized_distance[bottom_indices]) / np.sum(normalized_distance[bottom_indices]**2)

    if math.isnan(a1):
        a1 = 1
    if math.isnan(a2):
        a2 = 1
    if math.isnan(a3):
        a3 = 1
    if math.isnan(a4):
        a4 = 1
    return a1, a2, a3, a4


In [9]:

def calculate_shape(max_width,max_height,flower_pixels): #max_width,max_height  of flower


    # Calculate the area of the flower
    flower_area = max_width * max_height * (flower_pixels / (max_width * max_height))

    # Calculate the uncovered area for each shape
    circle_radius = math.sqrt(flower_area / math.pi)
    circle_area = math.pi * circle_radius ** 2
    circle_uncovered_area = circle_area - flower_area

    triangle_side = math.sqrt((4 * flower_area) / (math.sqrt(3) * 3))
    triangle_area = (math.sqrt(3) / 4) * triangle_side ** 2
    triangle_uncovered_area = triangle_area - flower_area

    rectangle_width = math.sqrt(flower_area / 2)
    rectangle_height = 2 * rectangle_width
    rectangle_area = rectangle_width * rectangle_height
    rectangle_uncovered_area = rectangle_area - flower_area
    # min_uncovered_area = min(circle_uncovered_area, triangle_uncovered_area, rectangle_uncovered_area)

    # if min_uncovered_area == circle_uncovered_area:
    #     min_shape = "circle"
    # elif min_uncovered_area == triangle_uncovered_area:
    #     min_shape = "triangle"
    # else:
    #     min_shape = "rectangle"

    c= circle_area/flower_pixels
    t= triangle_area/flower_pixels
    r= rectangle_area/flower_pixels
    if math.isnan(c):
        c = 1
    if math.isnan(r):
        r = 1

    if math.isnan(t):
        t = 1
    return  c, t, r

In [10]:
def calculate_column_averages(feature_vectors):
    # Convert the list of feature vectors to a numpy array
    feature_vectors_array = np.array(feature_vectors)

    # Calculate the mean of each column
    column_averages = np.mean(feature_vectors_array, axis=0)

    # Convert the numpy array to a list and return it
    return column_averages.tolist()

In [11]:
def get_features(clusters,points_num, image_width, image_height):
    features_vectors =[]
    for cluster in clusters:
        cluster_array = np.array(cluster)
        min_row = np.min(cluster_array[:, 3])
        max_row = np.max(cluster_array[:, 3])
        min_col = np.min(cluster_array[:, 4])
        max_col = np.max(cluster_array[:, 4])
        width = max_col-min_col
        height = max_row-min_row

        most_hue = mode(cluster_array[:, 0] )
        # features_vector = []
        hue_changes1,hue_changes2,hue_changes3,hue_changes4 = calculate_hue_change_coefficients(cluster, image_width, image_height)
        circle,traingle,rectangle = calculate_shape(width,height,points_num) #width,height
        features_vector = [most_hue,hue_changes1,hue_changes2,hue_changes3,hue_changes4,circle,traingle,rectangle]
        features_vectors.append(features_vector)

    return calculate_column_averages(features_vectors)

In [12]:


def MatrixCal(true_labels,predicted_labels):

# Create the confusion matrix
   confusion_mat = confusion_matrix(true_labels, predicted_labels)

# Create a pandas DataFrame for better visualization
   confusion_df = pd.DataFrame(confusion_mat, index=['True1', 'True2', 'True3', 'True4', 'True5', 'True6', 'True7'],
                             columns=['Pred1', 'Pred2', 'Pred3', 'Pred4', 'Pred5', 'Pred6', 'Pred7'])

   print("Confusion Matrix:")
   print(confusion_df)

# Calculate the precision, recall, f1-score, and accuracy
   precision = precision_score(true_labels, predicted_labels, average='weighted')
   recall = recall_score(true_labels, predicted_labels, average='weighted')
   f1 = f1_score(true_labels, predicted_labels, average='weighted')
   accuracy = accuracy_score(true_labels, predicted_labels)

   print("\nPrecision:", precision)
   print("Recall:", recall)
   print("F1-score:", f1)
   print("Accuracy:", accuracy)

   return confusion_mat

In [13]:
def delete_mistakes(data, true_labels, predicted_labels, delete_percentage=0.2, random_seed=42):
    confusion_mat = MatrixCal(true_labels, predicted_labels)
    
    misclassified_indices = np.where(true_labels != predicted_labels)[0]
    
    # Calculate mistake score for each misclassified sample
    mistake_scores = confusion_mat[predicted_labels[misclassified_indices], 1] + confusion_mat[predicted_labels[misclassified_indices], 2]
    
    # Sort indices based on mistake scores in descending order
    sorted_indices = np.argsort(mistake_scores)[::-1]
    
    # Calculate the number of samples to delete
    num_samples_to_delete = int(delete_percentage * len(misclassified_indices))
    
    # Select the top 20% of samples with the highest mistake scores to delete
    samples_to_delete = sorted_indices[:num_samples_to_delete]
    
    # Delete selected samples from the original data and labels
    cleaned_data = np.delete(data, misclassified_indices[samples_to_delete], axis=0)
    cleaned_true_labels = np.delete(true_labels, misclassified_indices[samples_to_delete])
    
    return cleaned_data, cleaned_true_labels

In [14]:


def get_accuracy2(features_vector,labels):


    # random_indices = random.sample(range(len(images2)), num)
    # images = [images2[i] for i in random_indices]
    # labels = [labels2[i] for i in random_indices]
    # features = [features2[i] for i in random_indices]

        # Replace NaN values with integers based on the other lists in features_vector
        # for j in range(len(v)):
        #     if np.isnan(v[j]):
        #         v[j] = int(np.nanmean(features_vector_arr[:, j]))  # Replace NaN with the mean of the non-NaN values for that feature

        # features_vector.append(v)


    y_test,y_pred =classify(features_vector, labels)

    MatrixCal(y_test,y_pred)

In [ ]:
# select faeture



scaler = StandardScaler()
X_normalized = scaler.fit_transform(X)

# Step 3: Calculate Correlation Matrix
correlation_matrix = X.corr()

# Step 4: Evaluate Feature Accuracy and Correlation
feature_scores = []
for feature in X.columns:
    X_single_feature = X[[feature]]
    X_train, X_test, y_train, y_test = train_test_split(X_single_feature, y, test_size=0.2, random_state=42)

    # Train RandomForestClassifier
    clf = RandomForestClassifier(random_state=42)
    clf.fit(X_train, y_train)

    # Predict and evaluate accuracy
    y_pred = clf.predict(X_test)
    accuracy = clf.score(X_test, y_test)

    # Calculate feature correlation with other features
    feature_corr = np.abs(correlation_matrix[feature].drop(feature)).min()

    feature_scores.append((feature, accuracy, feature_corr))

# Step 5: Calculate F1 Score based on Scores
best_f1_score = 0
selected_features = None

for feature, accuracy, feature_corr in feature_scores:
    # Calculate a combined score based on accuracy and inverse of correlation
    combined_score = accuracy / (1 + feature_corr)

    # If you have a specific weighting preference, you can adjust the formula

    if combined_score > best_f1_score:
        best_f1_score = combined_score
        selected_features = [feature]

# Print the selected features and the best F1 score
print("Selected Features:", selected_features)
print("Best F1 Score:", best_f1_score)

In [ ]:
get_accuracy2()


In [14]:
def remove_cluster(clusters,del_index, rows, cols, pixels_num):

    remaining_clusters = list(clusters)

    # accuracies = []
    # for i, cluster in enumerate(remaining_clusters):
    if del_index >= 0:
      result = remaining_clusters[:del_index] + remaining_clusters[del_index+1:]
    else: result = remaining_clusters
    # features_vector = get_features(,pixels_num,rows, cols )
    features_vector = get_features(result,pixels_num,rows, cols )
        # _ ,_ , accuracy = classify(features,labels)
        # accuracies.append(accuracy)

    # max_accuracy_index = accuracies.index(max(accuracies))
    # print("Removing cluster:", remaining_clusters[max_accuracy_index])
    # del remaining_clusters[max_accuracy_index]

    return features_vector , result


    # final_cluster_index = clusters.index(remaining_clusters[0])
    # print("Final cluster:", remaining_clusters[0] + ", Index:", final_cluster_index)
    # return final_cluster_index

In [15]:
def correct_feature_vector(matrix):
    # Convert the 2D list to a NumPy array for easier manipulation
    array = np.array(matrix, dtype=float)

    # Iterate over columns
    for col in range(array.shape[1]):
        # Find indices of NaN values in the column
        nan_indices = np.isnan(array[:, col])

        # Calculate the column average excluding NaN values
        col_average = np.nanmean(array[:, col])

        # Replace NaN values with the column average
        array[nan_indices, col] = col_average

    # Convert the NumPy array back to a 2D list
    result_matrix = array.tolist()

    return result_matrix

In [23]:
def find_flower_cluster(features,labels):
    # images_flower_index = []
    images_cluster = []
    images_width = []
    images_height = []
    images_points_num = []
    images_pixel_lables = []
    cur_feature_vector = []
    cur_clusters = []
    for i, image in enumerate(features):
      pixel_labels,clusters,width,height,points_num= extract_flower_parts(features[i])
      images_width.append(width)
      images_cluster.append(clusters)
      images_height.append(height)
      images_points_num.append(points_num)
      images_pixel_lables.append(pixel_labels)

    cluster_num = len(images_cluster[0])
    images_cluster_origin = images_cluster # copy original clusters

    for i in range(0,len(features)):
        fv,c = remove_cluster(images_cluster[i],-1,images_width[i], images_height[i], images_points_num[i])
        cur_feature_vector.append(fv)
        cur_clusters.append(c)
    # print(cur_feature_vector)
    _, _, pre_accuracy = classify(cur_feature_vector,labels)
    print(pre_accuracy)
    
    print("----------------------")
    del_index = 0
    state = ""
    while cluster_num > 1 :
      print(f"del_index is {del_index}")
      print(f"number clusters {cluster_num}")
      for i in range(0,len(features)):
        cur_feature_vector[i],cur_clusters[i] = remove_cluster(images_cluster[i],del_index,images_width[i], images_height[i], images_points_num[i])
      # print(len(cur_feature_vector))
      _, _, cur_accuracy = classify(cur_feature_vector,labels)
      print(len(cur_clusters[0]))
      if cur_accuracy >= pre_accuracy:
        cluster_num -= 1
        images_cluster = cur_clusters
        state += str(del_index)
        del_index = 0
      else :
        del_index = + 1

      if del_index == cluster_num:
        for i in range(0,len(features)):
          cur_feature_vector[i],cur_clusters[i] = remove_cluster(images_cluster[i],0,images_width[i], images_height[i], images_points_num[i])
        images_cluster = cur_clusters
        _, _, cur_accuracy = classify(cur_feature_vector,labels)
        state += str(0)
      
      pre_accuracy = cur_accuracy
      print("----------------------")

    
    print(state)
    # for i in range(0,len(features)) :
    #   images_flower_index.append(images_cluster_origin.index(images_cluster[i][0]))
    if state == '00':
      images_flower_index = [2 for _ in range(len(features))]
    elif state == '01':
      images_flower_index = [1 for _ in range(len(features))]
    elif state == '10':
      images_flower_index = [2 for _ in range(len(features))]
    elif state == '11':
      images_flower_index = [0 for _ in range(len(features))]
    elif state == '20':
      images_flower_index = [1 for _ in range(len(features))]
    else: #21
      images_flower_index = [0 for _ in range(len(features))]

    _, _, cur_accuracy = classify(cur_feature_vector,labels)
    print(f"final accuray {cur_accuracy}")

    return images_cluster_origin,images_flower_index,images_width,images_height , images_pixel_lables

In [25]:
def test_finding_flower(num):
    images2, labels2, features2 = load_dataset()

    random_indices = random.sample(range(len(images2)), num)

    images = [images2[i] for i in random_indices]
    labels = [labels2[i] for i in random_indices]
    features = [features2[i] for i in random_indices]

    images_cluster,images_flower_index,images_width,images_height, images_pixel_lables = find_flower_cluster(features,labels)
    for j in range(0,num):
        print(j)
        show_image(images[j])
        show_cluster_image2(images_pixel_lables[j],images_cluster[j][images_flower_index[j]][0][5],images_width[j],images_height[j])





In [26]:
test_finding_flower(150)

c:\Users\mashadservice\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\mashadservice\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\mashadservice\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\mashadservice\AppData\Local\Programs\Python\Python31

In [ ]:
def all():
    images2, labels2, features2 = load_dataset()

    random_indices = random.sample(range(len(images2)), 50)

    images = [images2[i] for i in random_indices]
    labels = [labels2[i] for i in random_indices]
    features = [features2[i] for i in random_indices]

    features_vector = []
    for i in range(0,len(images)):
        # show_image(images[i])
        pixel_labels2,clusters,image_width, image_height ,points_num= extract_flower_parts(features[i])
        flower_index = find_flower_cluster(clusters,image_width, image_height,points_num)